### Notes 

T5 Paper: https://arxiv.org/pdf/1910.10683.pdf

T5 Tokenizer: https://github.com/huggingface/transformers/blob/master/src/transformers/tokenization_t5.py

Important Tasks: https://docs.google.com/document/d/1weIZM6QTlnitpPQmpg-WeV2RW70TnYmDuogBQPr5mB0/edit

In [1]:
#installation step
!pip install transformers
!pip install t5
!pip install sentencepiece
!pip install utils
#creating the folders 
!mkdir data/
!mkdir data/AD_NMT-master
!mkdir data/train/
!mkdir data/test/
!mkdir data/val/
!mkdir data/vocab/
!mkdir data/model/
#fetching the pkl files
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW' -O data/AD_NMT-master/english-Arabic-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V8_tp8ZlWUYaX7QQL46t0uSRNrVehSf1' -O data/AD_NMT-master/english-Arabic-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V7X0qtuDIyjTHY0wh-ZNoVwsiF4lId2e' -O data/AD_NMT-master/english-Arabic-train.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UzL4cOWTMCee83KBUh2QO_H62AFVpDQV' -O data/AD_NMT-master/LAV-MSA-2-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UpfCbkxhztof7dvNjeAs1bHjD4SER6h3' -O data/AD_NMT-master/LAV-MSA-2-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UlAZGtYsSfXzK7hrC_PbxQFqTSXD0DMw' -O data/AD_NMT-master/LAV-MSA-2-train.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UjDX7cCG2S23SPfSHxSPdVayMTxB5Y16' -O data/AD_NMT-master/Magribi_MSA-both.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UaVWIqRXo0rxuxDF4KArA4bEK1TaLX3l' -O data/AD_NMT-master/Magribi_MSA-test.pkl
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UYvlhdYAdfa4riP_4hn3-IEVd1ZUXVTQ' -O data/AD_NMT-master/Magribi_MSA-train.pkl

     |████████████████████████████████| 573kB 2.6MB/s 
     |████████████████████████████████| 1.0MB 6.9MB/s 
     |████████████████████████████████| 890kB 14.0MB/s 
     |████████████████████████████████| 3.7MB 19.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=1679606b3174adaa10bbbb0f797fcaebe688ce895bd982d65c7e2aeda0dbe470
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses
     |████████████████████████████████| 143kB 2.7MB/s 
     |████████████████████████████████| 296kB 50.2MB/s 
     |████████████████████████████████| 8.3MB 52.4MB/s 
     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 3.3MB 46.9MB/s 
     |████████████████████████████████| 421.8MB 21kB/s 
     |████████████████████████████████| 17.1MB 250kB/s 
     |████████████████████████████████| 3.9MB 50.2MB/s 
     |█████████████████████████████

--2020-04-28 12:15:36--  https://docs.google.com/uc?export=download&id=1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW
Resolving docs.google.com (docs.google.com)... 74.125.31.113, 74.125.31.100, 74.125.31.139, ...
Connecting to docs.google.com (docs.google.com)|74.125.31.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-10-2s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/gd9qrpsqa0ur41kg4higb39n6cip8utg/1588076100000/16970776037313924126/*/1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW?e=download [following]
--2020-04-28 12:15:36--  https://doc-10-2s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/gd9qrpsqa0ur41kg4higb39n6cip8utg/1588076100000/16970776037313924126/*/1V9crCmqvgQcv0Sx2MCNWB9AET2j6M6FW?e=download
Resolving doc-10-2s-docs.googleusercontent.com (doc-10-2s-docs.googleusercontent.com)... 173.194.216.132, 2607:f8b0:400c:c12::84
Connecting to doc-10-2s-docs.googleusercontent.com (doc-10-2s-docs

In [0]:
#James Chartouni
#Joey Park
#Raef Khan

import torch
import pandas as pd
import numpy as np
import pickle
import os, io, glob
import functools

import sentencepiece as spm

import transformers
import t5
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split

In [0]:
VOCAB_SIZE = 5000

In [10]:
ls data/AD_NMT-master

english-Arabic-both.pkl   LAV-MSA-2-both.pkl   Magribi_MSA-both.pkl
english-Arabic-test.pkl   LAV-MSA-2-test.pkl   Magribi_MSA-test.pkl
english-Arabic-train.pkl  LAV-MSA-2-train.pkl  Magribi_MSA-train.pkl


In [0]:
file_path = 'data/AD_NMT-master/'

with open(file_path + "english-Arabic-train.pkl", 'rb') as handle:
    data_English_MSA_trainval = pickle.load(handle)

with open(file_path + "english-Arabic-test.pkl", 'rb') as handle:
    data_English_MSA_test = pickle.load(handle)

with open(file_path + "english-Arabic-both.pkl", 'rb') as handle:
    data_English_MSA_both = pickle.load(handle) 




with open(file_path + "LAV-MSA-2-train.pkl", 'rb') as handle:
    data_LAV_MSA_trainval = pickle.load(handle) 

with open(file_path + "LAV-MSA-2-test.pkl", 'rb') as handle:
    data_LAV_MSA_test = pickle.load(handle) 

with open(file_path + "LAV-MSA-2-both.pkl", 'rb') as handle:
    data_LAV_MSA_both = pickle.load(handle) 




with open(file_path + "Magribi_MSA-train.pkl", 'rb') as handle:
    data_Magribi_MSA_trainval = pickle.load(handle) 
    
with open(file_path + "Magribi_MSA-test.pkl", 'rb') as handle:
    data_Magribi_MSA_test = pickle.load(handle) 

with open(file_path + "Magribi_MSA-both.pkl", 'rb') as handle:
    data_Magribi_MSA_both = pickle.load(handle) 
    

In [12]:
#few dataset examples
print(data_English_MSA_both[0:5])
print(data_LAV_MSA_both[0:5])

#print length of train + val dataset, print length of both (train + val) + test
print(len(data_English_MSA_trainval))
print(len(data_English_MSA_both))

[['Tom was also there', 'كان توم هنا ايضا'], ['That old woman lives by herself', 'تلك المراة العجوز تسكن بمفردها'], ['He went abroad for the purpose of studying English', 'سافر خارج البلد ليتعلم الانجليزية'], ['There is a fork missing', 'هناك شوكة ناقصة'], ["I don't know this game", 'لا اعرف هذه اللعبة']]
[['لا انا بعرف وحدة راحت ع فرنسا و معا شنتا حطت فيها الفرش', 'لا اعرف واحدة ذهبت الى فرنسا و لها غرفة و ضعت فيها الافرشة'], ['روح بوشك و فتول عاليسار', 'اذهب تقدم و استدر يسارا'], ['لا لا لازم انه يكون عندك موضوع ما في اشي', ' لا لا يجب ان يكون لديك موضوع هذا ضروري'], ['اوعي تبعدي من هون بلاش تضيعي ', 'لا تبتعد عن هنا حتى لا تفقد الطريق '], ['قصدي صراحة يما انا كمان كرهته من يوم ما عملتيه زي ما بتعمله خالتي كرهته و صرت ما باطيقه بالمرة', 'اقصد صراحة يا امي انا ايضا كرهته من يوم حضرته مثلما تحضره خالتي كرهته و اصبحت لا اطيقه ابدا']]
9000
10001


## Prepare Datasets

example: https://iwslt2010.fbk.eu/node/32/

We need to take our training and test sets from the pkl files and create new .txt files that are formatted so that the standard torchtext Dataset class can read them

Data format:
each line consists of three fields divided by the character '\'
sentences consisting of words divided by single spaces
format: <SENTENCE_ID>\<PARAPHRASE_ID>\<TEXT>
Field_1: sentence ID
Field_2: paraphrase ID
Field_3: MT develop sentence / reference translation
Text input example:
DEV_001\01\This is the first develop sentence.
DEV_002\01\This is the second develop sentence.
Reference translation example:
DEV_001\01\1st reference translation for 1st input
DEV_001\02\2nd reference translation for 1st input
...
DEV_002\01\1st reference translation for 2nd input
DEV_002\02\2nd reference translation for 2nd input
...
Languages:
Arabic-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations
IWSLT05 testset: 506 sentences, 16 reference translations
IWSLT07 testset: 489 sentences, 6 reference translations
IWSLT08 testset: 507 sentences, 16 reference translations
French-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations
IWSLT05 testset: 506 sentences, 16 reference translations
Turkish-English
CSTAR03 testset: 506 sentences, 16 reference translations
IWSLT04 testset: 500 sentences, 16 reference translations


In [13]:
ls data/AD_NMT-master/

english-Arabic-both.pkl   LAV-MSA-2-both.pkl   Magribi_MSA-both.pkl
english-Arabic-test.pkl   LAV-MSA-2-test.pkl   Magribi_MSA-test.pkl
english-Arabic-train.pkl  LAV-MSA-2-train.pkl  Magribi_MSA-train.pkl


In [0]:
#splits the train dataset into train and validation sets, define test set as datafile
eng_msa_train, eng_msa_val = train_test_split(data_English_MSA_trainval, test_size=.2)
eng_msa_test = data_English_MSA_test

lav_msa_train, lav_msa_val = train_test_split(data_LAV_MSA_trainval, test_size=.2)
lav_msa_test = data_LAV_MSA_test

mag_msa_train, mag_msa_val = train_test_split(data_Magribi_MSA_trainval, test_size=.2)
mag_msa_test = data_Magribi_MSA_test

In [18]:
print(len(eng_msa_train))
print(len(eng_msa_val))

print(len(lav_msa_train))
print(len(lav_msa_val))

print(len(mag_msa_train))
print(len(mag_msa_val))

7200
1800
11044
2761
14188
3548


In [0]:
file_path = 'data/'

def list_to_csv(ds, src='en', trg='msa', datatype=''):
    src_formatted = datatype + '_' + src + '_' + trg + '.' + 'csv'
    
    with open(file_path + datatype + "/" + src_formatted, 'wt') as csv:
        for i, arr in enumerate(ds):
            csv.write(arr[0] + ',' + arr[1])

In [0]:
list_to_csv(eng_msa_train, 'eng', 'msa', 'train')
list_to_csv(eng_msa_val, 'eng', 'msa', 'val')
list_to_csv(eng_msa_test, 'eng', 'msa', 'test')

list_to_csv(lav_msa_train, 'lav', 'msa', 'train')
list_to_csv(lav_msa_val, 'lav', 'msa', 'val')
list_to_csv(lav_msa_test, 'lav', 'msa', 'test')

list_to_csv(mag_msa_train, 'mag', 'msa', 'train')
list_to_csv(mag_msa_val, 'mag', 'msa', 'val')
list_to_csv(mag_msa_test, 'mag', 'msa', 'test')

##Tensor Processing + Add to TaskRegistry

In [0]:
eng_msa_split_csv_path = {
    "train": "data/train/train_eng_msa.csv",
    "validation": "data/val/val_eng_msa.csv"
}

In [31]:
def translation_dataset_fn(split, shuffle_files=False):
  ds = tf.data.TextLineDataset(eng_msa_split_csv_path[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_default=["",""],
                        field_delim=",", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE
  )
  ds = ds.map(lambda *example: dict(zip(["source", "target"], example)) )
  return ds

for example in tfds.as_numpy(translation_dataset_fn("train").take(5)):
  print(example)

TypeError: ignored

In [0]:
#turn the ds of dictionaries and change the keys to inputs and targets that the model
def tranlation_preprocessor(ds):
  def to_inputs_and_targets(ex):
    return{
        "inputs": tf.strings.join("Translate English to Arabic: ",ex["source"]),
        "targets": ex["target"]
    }
  return ds.map(to_inputs_and_targets, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [0]:
t5.data.TaskRegistry.add(
    #name of the Task
    "arabic_translation",
    #Supply a function which returns a tf.data.Dataset
    dataset_fn=translation_dataset_fn
    splits=[]
)

## Build Vocabulary 

Sentence Piece Google Colab
https://github.com/google/sentencepiece/blob/master/python/sentencepiece_python_module_example.ipynb

In [0]:
!mkdir data/vocab
!mkdir data/model

In [0]:
"""
Testing:
Create a text file with all the vocab available from all sources for each language for SentencePiece to create a library 

TODO: implement bpemb model for MSA vocab/train SPM on larger datasets for dialects
"""

en_vocab = open("data/vocab/eng_vocab.txt", "wt")
msa_vocab = open("data/vocab/msa_vocab.txt", "wt")
lav_vocab = open("data/vocab/lav_vocab.txt", "wt")
mag_vocab = open("data/vocab/mag_vocab.txt", "wt")

MSA_text = ""
EN_text = ""

def create_vocab(file='', src='en_vocab', tgt='msa_vocab'):
  for line in file:
        src_sent = line[0]
        src_words = src_sent.split(" ")
        for count, word in enumerate(src_words):
            src.write(word)
        src.write("\n")
        
        tgt_sent = line[1]
        tgt_words = tgt_sent.split(" ")
        for count, word in enumerate(tgt_words):
            tgt.write(word)
        tgt.write("\n")

create_vocab(data_English_MSA_both, en_vocab, msa_vocab)
create_vocab(data_LAV_MSA_both, lav_vocab, msa_vocab)
create_vocab(data_Magribi_MSA_both, mag_vocab, msa_vocab)

en_vocab.close()
msa_vocab.close()
lav_vocab.close()
mag_vocab.close()

In [0]:
#spm.SentencePieceTrainer.train('--input=data/vocab/eng_vocab.txt,data/vocab/msa_vocab.txt,data/vocab/lav_vocab.txt,data/vocab/mag_vocab.txt --model_prefix=data/model/spm --vocab_size=' + str(VOCAB_SIZE))
spm.SentencePieceTrainer.train('--input=data/vocab/eng_vocab.txt --model_prefix=data/model/eng --vocab_size=' + str(VOCAB_SIZE))
spm.SentencePieceTrainer.train('--input=data/vocab/msa_vocab.txt --model_prefix=data/model/msa --vocab_size=' + str(VOCAB_SIZE))
spm.SentencePieceTrainer.train('--input=data/vocab/lav_vocab.txt --model_prefix=data/model/lav --vocab_size=' + str(VOCAB_SIZE))
spm.SentencePieceTrainer.train('--input=data/vocab/mag_vocab.txt --model_prefix=data/model/mag --vocab_size=' + str(VOCAB_SIZE))

In [0]:
ls data/model

In [0]:
sp = spm.SentencePieceProcessor()
sp.load('data/model/eng.model')

In [0]:
print(sp.encode_as_pieces('This is a test'))
print(sp.encode_as_ids('This is a test'))

## Model Training

https://huggingface.co/transformers/model_doc/t5.html#training

https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html

In [0]:
count = 0

for x in train_iterator:
    print(x)
    print(x.src)
    print(x.trg)
    #print(spacy_en_tokenizer.decode(x.src))
    count += 1
    
print(count)

In [0]:
PAD_IDX = TRG.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [0]:
def train(model: nn.Module,
          iterator: BucketIterator,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, batch in enumerate(iterator):

        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


In [0]:
def evaluate(model: nn.Module,
             iterator: BucketIterator,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing?

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [0]:
#declare model
config = T5Config(vocab_size=VOCAB_SIZE)
model = T5Model(config)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [0]:
import time
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

In [0]:
def trainIters(model, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    optimizer = AdamW.SGD(model.parameters(), lr=learning_rate)

    criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX) #CHECK IF THE BEST LOSS FUNCTION

    for iter in range(1, n_iters + 1):
        

        loss = train(decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [0]:
model = 
n_iteres = 10
trainIters(model, n_iters)

## Model Inference

https://github.com/huggingface/transformers/blob/master/examples/translation/t5/evaluate_wmt.py

In [0]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

In [0]:
from tqdm import tqdm
def generate_translations(lns, output_file_path, model_size, batch_size, device):
    model = T5ForConditionalGeneration.from_pretrained(model_size)
    model.to(device)
    #returned to using T5 tokenizer
    tokenizer = tokenizer

    task_specific_params = model.config.task_specific_params

    with Path(output_file_path).open("w") as output_file:
        for batch in tqdm(list(chunks(lns, batch_size))):
        #modify to have prefix 'translate English to Arabic: ...'
            batch = [ text for text in batch]
            
            dct = tokenizer.batch_encode_plus(batch, max_length=512, return_tensors="pt", pad_to_max_length=True)

            input_ids = dct["input_ids"].to(device)
            attention_mask = dct["attention_mask"].to(device)

            translations = model.generate(input_ids=input_ids, attention_mask=attention_mask)
            dec = [
                tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in translations
            ]

            for hypothesis in dec:
                output_file.write(hypothesis + "\n")

In [0]:
from pathlib import Path
def calculate_bleu_score(output_lns, refs_lns, score_path):
    bleu = corpus_bleu(output_lns, [refs_lns])
    result = "BLEU score: {}".format(bleu.score)
    with Path(score_path).open("w") as score_file:
        score_file.write(result)

In [0]:
!mkdir data/output/
!touch data/output/translated.msa
!touch data/output/score_eng_msa.txt

In [0]:
#got rid of the argparse and just put it in as params
def run_generate(model_size='t5-base', input_path='data/train/train_eng_msa.eng', output_path='data/output/translated.msa', ref_path='data/train/train_eng_msa.eng', score_path='data/output/score_eng_msa.txt', batch_size=BATCH_SIZE):
    device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")

    # Read input lines into python
    with open(input_path, "r") as input_file:
        input_lns = [x.strip() for x in input_file.readlines()]

    generate_translations(input_lns, output_path, model_size, batch_size, device)

    # Read generated lines into python
    with open(output_path, "r") as output_file:
        output_lns = [x.strip() for x in output_file.readlines()]

    # Read reference lines into python
    with open(ref_path, "r") as reference_file:
        refs_lns = [x.strip() for x in reference_file.readlines()]

    calculate_bleu_score(output_lns, refs_lns, args.score_path)

In [0]:
run_generate()